# Importación de módulos requeridos

In [1]:
from pyspark.sql import SparkSession, functions as f
from blackops.crawlers.wallapop.functions import fetch_api
import json
from datetime import date, timedelta
import random

In [2]:
random.seed(42)

# Inicialización de la sesión de Spark

En este caso no estamos utilizando un clúster, sino que haremos uso de una arquitectura local. El propio Jupyter Notebook ejercerá como Driver, como Master y como Ejecutor de las tareas.

Adicionalmente, estamos instalando dependencias externas como la librería Delta, que incorpora utilidades muy importantes para el manejo de las tablas en nuestro catálogo de datos

In [3]:
spark = (
    SparkSession.Builder()
    .master("local[*]")
    .config(
        map={
            "spark.driver.memory": "8g",
            "spark.jars.packages": "io.delta:delta-spark_2.12:3.2.0",
            "spark.sql.repl.eagerEval.enabled": "true",
            "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
            "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
            "spark.sql.catalogImplementation": "hive",
        }
    )
    .getOrCreate()
)

24/09/25 18:28:15 WARN Utils: Your hostname, dadiego resolves to a loopback address: 127.0.1.1; using 192.168.199.128 instead (on interface eth1)
24/09/25 18:28:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/dadiego/projects/esic-bigdata-iv-blackops/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dadiego/.ivy2/cache
The jars for the packages stored in: /home/dadiego/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a2bfb13e-15fa-4c73-b7bb-ed0ddb58e310;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 164ms :: artifacts dl 15ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |  

# Creación de un DataFrame de Spark

En Spark podemos crear directamente un Dataframe a partir de una lista de datos, o bien de un Dataframe de pandas. Para ello se puede utilizar el método `spark.createDataFrame`.

In [4]:
# Podemos especificar el esquema del DataFrame usando una cadena de texto
schema = "id INT, nombre STRING, edad INT, salario FLOAT, es_empleado BOOLEAN, fecha_contratacion DATE, departamento STRING"

# Crear una lista de datos ficticios
nombres = [
    "Juan",
    "María",
    "Pedro",
    "Ana",
    "Luis",
    "Carla",
    "Miguel",
    "Sara",
    "David",
    "Laura",
]
departamentos = ["Ventas", "Marketing", "Finanzas", "IT", "RRHH"]

data = [
    (
        i,  # id
        random.choice(nombres),  # nombre
        random.randint(22, 60),  # edad
        round(random.uniform(20000, 80000), 2),  # salario
        random.choice([True, False]),  # es_empleado
        date(2024, 10, 1)
        - timedelta(days=random.randint(0, 3650)),  # fecha_contratacion
        random.choice(departamentos),  # departamento
    )
    for i in range(1, 21)  # Genera 20 registros
]

# Crear el DataFrame usando el esquema en string
df = spark.createDataFrame(data, schema)

display(df)

id,nombre,edad,salario,es_empleado,fecha_contratacion,departamento
1,María,23,64493.03,true,2022-04-01,Marketing
2,María,56,25216.33,false,2024-05-24,Ventas
3,María,35,33959.65,true,2018-06-17,Marketing
4,David,48,33226.44,false,2015-09-04,Ventas
5,Pedro,49,40415.03,true,2022-05-04,Finanzas
6,María,27,42795.64,false,2015-04-01,Finanzas
7,Laura,38,68427.7,false,2018-09-27,Ventas
8,Miguel,27,53122.44,false,2018-04-12,Marketing
9,María,24,59675.8,false,2023-11-10,Marketing
10,María,46,36678.42,false,2022-12-05,Finanzas


# Operaciones de transformación

La sintaxis de Spark es muy similar a la del lenguaje SQL, de hecho, admite la introducción de comandos SQL para realizar las transformaciones de los datos.

### Select

La operación más sencilla consiste en seleccionar un subconjunto de los datos

In [5]:
df.select("id", "nombre")

id,nombre
1,María
2,María
3,María
4,David
5,Pedro
6,María
7,Laura
8,Miguel
9,María
10,María


Al igual que en SQL estándar, podemos no solo seleccionar unas columnas sino aplicarles alguna función sencilla dentro del propio comando SELECT, y renombrarlas utilizando un alias

In [6]:
df.select("id", f.upper("nombre").alias("nombre_en_mayusculas"))

id,nombre_en_mayusculas
1,MARÍA
2,MARÍA
3,MARÍA
4,DAVID
5,PEDRO
6,MARÍA
7,LAURA
8,MIGUEL
9,MARÍA
10,MARÍA


### WithColumn

Podemos añadir campos nuevos derivados utilizando el método `withColumn`

In [7]:
df.withColumn("año_nacimiento", f.year(f.current_date()) - f.col("edad"))

id,nombre,edad,salario,es_empleado,fecha_contratacion,departamento,año_nacimiento
1,María,23,64493.03,true,2022-04-01,Marketing,2001
2,María,56,25216.33,false,2024-05-24,Ventas,1968
3,María,35,33959.65,true,2018-06-17,Marketing,1989
4,David,48,33226.44,false,2015-09-04,Ventas,1976
5,Pedro,49,40415.03,true,2022-05-04,Finanzas,1975
6,María,27,42795.64,false,2015-04-01,Finanzas,1997
7,Laura,38,68427.7,false,2018-09-27,Ventas,1986
8,Miguel,27,53122.44,false,2018-04-12,Marketing,1997
9,María,24,59675.8,false,2023-11-10,Marketing,2000
10,María,46,36678.42,false,2022-12-05,Finanzas,1978


### Filter

Podemos filtrar los datos de acuerdo a alguna condición especificada. Por ejemplo, queremos obtener únicamente los datos de los empleados

In [10]:
df.filter(f.col("es_empleado") == True)

id,nombre,edad,salario,es_empleado,fecha_contratacion,departamento
1,María,23,64493.03,true,2022-04-01,Marketing
3,María,35,33959.65,true,2018-06-17,Marketing
5,Pedro,49,40415.03,true,2022-05-04,Finanzas
11,Carla,35,60210.51,true,2017-12-02,Marketing
13,Ana,42,70571.12,true,2022-03-08,Ventas
14,Carla,47,36064.45,true,2018-05-23,Finanzas
17,Miguel,59,43964.03,true,2023-03-15,RRHH
18,Sara,27,65346.93,true,2023-01-14,Marketing
20,David,38,78264.7,true,2017-02-14,Ventas


### Agrupaciones

Utilizando el comando group by, poodemos agrupar nuestro dataset según los valores de una o varias columnas y posteriormente realizar una operación de agregación sobre cada conjunto, para así obtener estadísticas descriptivas de nuestros datos.

Por ejemplo, podemos obtener el número de empleados en marketing 

In [13]:
df.groupBy("departamento").agg(f.sum("salario").alias("salario_total"))

departamento,salario_total
Finanzas,199691.453125
Ventas,275706.294921875
Marketing,336808.36328125
RRHH,138467.521484375
IT,23811.66015625
